In [ ]:
import requests
import pandas as pd
from geopy.geocoders import Nominatim
import pytz
from datetime import datetime
import numpy as np
from IPython.display import display, clear_output, Markdown


date = datetime(2018, 11, 1, 0, 0, 0, tzinfo=pytz.utc)
places = ['Bottrop, DE']
#places = ['Bottrop, DE', 'Vantaa, FI', 'Bologna, IT', 'Hasselt, BE', 'Surrey, UK']

devices_api = 'https://api.smartcitizen.me/v0/devices/'

In [ ]:
def devices(place):

    geolocator = Nominatim(user_agent="iSCAPE")

    location = geolocator.geocode(place)

    location_str = str(location.latitude) + ',' + str(location.longitude)

    params = {'near': location_str, 'within': '10000'}

    req = requests.get(devices_api, params=params).json()
    
    # for item in req:
    #     print item
    devices = pd.DataFrame(req)
    devices['iscape_site'] = place
    devices['last_reading_at'] = pd.to_datetime(devices['last_reading_at'])
    devices = devices[devices['last_reading_at'] > date.isoformat()]
    devices = devices[devices['added_at'] > date.isoformat()]
    
    # Just for the testing
    devices.kit[0]['id'] = 21
    devices.kit[3]['id'] = 19
    
    # If we are looking for the 20, it should give us everything but 0 and 3
    # Probably this could be done via list comprehension but it's clearer this way
    # The problem comes with the dict within the pandas series, that it's not directly indexable from pandas (same sintax)
    list_kits = list()
    for item in devices.kit:
        if item['id'] == 20: list_kits = devices[devices.kit == item].index
    
    # Filter devices
    devices = devices.loc[list_kits]

    devices['total_readings'] = devices.apply(lambda row: readings(row), axis=1)

    return devices

def readings(row):

    params = {'from': date.isoformat(), 'to': datetime.now().isoformat(), 'rollup': '60s', 'sensor_id': 10 }

    req = requests.get(devices_api + str(row['id']) + '/readings', params=params).json()

    readings = req['readings']

    total_readings = len(readings)

    return total_readings

In [ ]:
## Remove lines 21-23 in the function devices (it's just an example)
sites = []
for place in places:
    
    sites.append(devices(place))
    
sites = pd.concat(sites)

display(sites)   